# Cloning repository and installing dependencies

In [1]:
!git clone https://github.com/pinellolab/DNA-Diffusion.git  && cd DNA-Diffusion && uv sync

Streaming output truncated to the last 5000 lines.
triton     ------------------------------ 95.65 MiB/149.25 MiB
nvidia-cusparselt-cu12 ------------------------------ 93.71 MiB/149.52 MiB
nvidia-cusolver-cu12 ------------------------------ 93.72 MiB/150.90 MiB
nvidia-cufft-cu12 ------------------------------ 94.08 MiB/190.95 MiB
nvidia-nccl-cu12 ------------------------------ 93.60 MiB/191.99 MiB
nvidia-cusparse-cu12 ------------------------------ 94.11 MiB/206.53 MiB
nvidia-cublas-cu12 ------------------------------ 93.89 MiB/374.93 MiB
nvidia-cudnn-cu12 ------------------------------ 94.56 MiB/544.54 MiB
⠼ Preparing packages... (161/171)
mkdocs-material ------------------------------ 2.11 MiB/8.30 MiB
triton     ------------------------------ 95.81 MiB/149.25 MiB
nvidia-cusparselt-cu12 ------------------------------ 94.18 MiB/149.52 MiB
nvidia-cusolver-cu12 ------------------------------ 94.12 MiB/150.90 MiB
nvidia-cufft-cu12 ------------------------------ 94.08 MiB/190.95 MiB
nvidi

In [5]:
%cd DNA-Diffusion

/content/DNA-Diffusion


# Downloading checkpoint from HuggingFace

In [6]:
!wget 'https://huggingface.co/ssenan/DNA-Diffusion/resolve/main/DNA-Diffusion.safetensors' -O checkpoints/DNA-Diffusion.safetensors

--2025-06-25 00:21:24--  https://huggingface.co/ssenan/DNA-Diffusion/resolve/main/DNA-Diffusion.safetensors
Resolving huggingface.co (huggingface.co)... 18.164.174.118, 18.164.174.55, 18.164.174.23, ...
Connecting to huggingface.co (huggingface.co)|18.164.174.118|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://cdn-lfs-us-1.hf.co/repos/e1/d4/e1d4c85e41464c38bc7651c3008e77acf461d72b740fdbf0cb82467b99dded45/6fde776215af0789c55982724c22fd2710ed47cdf9651e83c6343720be6f8270?response-content-disposition=inline%3B+filename*%3DUTF-8%27%27DNA-Diffusion.safetensors%3B+filename%3D%22DNA-Diffusion.safetensors%22%3B&Expires=1750814484&Policy=eyJTdGF0ZW1lbnQiOlt7IkNvbmRpdGlvbiI6eyJEYXRlTGVzc1RoYW4iOnsiQVdTOkVwb2NoVGltZSI6MTc1MDgxNDQ4NH19LCJSZXNvdXJjZSI6Imh0dHBzOi8vY2RuLWxmcy11cy0xLmhmLmNvL3JlcG9zL2UxL2Q0L2UxZDRjODVlNDE0NjRjMzhiYzc2NTFjMzAwOGU3N2FjZjQ2MWQ3MmI3NDBmZGJmMGNiODI0NjdiOTlkZGVkNDUvNmZkZTc3NjIxNWFmMDc4OWM1NTk4MjcyNGMyMmZkMjcxMGVkNDdjZGY5NjUxZTgzYzYzNDM3M

# Generate sequences

In [8]:
!uv run sample_hf.py sampling.number_of_samples=10 sampling.sample_batch_size=5

model:
  _target_: src.dnadiffusion.models.unet.UNet
  dim: 200
  channels: 1
  dim_mults:
  - 1
  - 2
  - 4
  resnet_block_groups: 4
data:
  _target_: src.dnadiffusion.data.dataloader.get_dataset
  data_path: data/K562_hESCT0_HepG2_GM12878_12k_sequences_per_group.txt
  saved_data_path: data/encode_data.pkl
  load_saved_data: true
  debug: false
diffusion:
  _target_: src.dnadiffusion.models.diffusion.Diffusion
  timesteps: 50
  beta_start: 0.0001
  beta_end: 0.2
sampling:
  checkpoint_path: checkpoints/DNA-Diffusion.safetensors
  sample_batch_size: 5
  number_of_samples: 10

(<src.dnadiffusion.data.dataloader.SequenceDataset object at 0x7d96c41dfa10>, <src.dnadiffusion.data.dataloader.SequenceDataset object at 0x7d96e409d760>, [1, 2, 3, 4], {1: 'GM12878_ENCLB441ZZZ', 2: 'HepG2_ENCLB029COU', 3: 'K562_ENCLB843GMH', 4: 'hESCT0_ENCLB449ZZZ'})
Loading checkpoint
Sending model to device
Generating 10 samples for cell GM12878_ENCLB441ZZZ
100% 2/2 [00:09<00:00,  4.77s/it]
Generating 10 sample

# Display generated sequences

In [9]:
import os
import subprocess

def display_sequences(output_dir="data/outputs"):
    if not os.path.isdir(output_dir):
        print(f"Error: Directory '{output_dir}' not found.")
        return

    print(f"Displaying sequences from: {output_dir}\n")

    for filename in sorted(os.listdir(output_dir)):
        filepath = os.path.join(output_dir, filename)

        if os.path.isfile(filepath) and "gitkeep" not in filepath:
          cell_type = filename.split('_')[0]
          print(f"--- Cell Type: {cell_type} ({filename}) ---")
          result = subprocess.run(['cat', filepath], capture_output=True, text=True, check=True)
          print(result.stdout)
          print("-" * (len(cell_type) + 18) + "\n")

display_sequences()

Displaying sequences from: data/outputs

--- Cell Type: GM12878 (GM12878_ENCLB441ZZZ.txt) ---
GTATATACGACTGTGTTTACATTTTTTACACGCACGCTAATTCAATATAACTCACACTCACGCCGCTTATACGGATAACAATACACGTACACAATTTACATTATTCTACACGGGGGCGCGACACACACACGTACACCCAACACGTTATACACCCCGCCACAGTATATACGCACAGCGCGTGTACACCAGAGATAGATTAA
CGCGCGCACCCCCTATCTACGCACACGGACACGTCGCTACACACACATAACAAAATTTACACACTCGAAAAGCGGGCACACCCGTCTAGTAAAACCGCGCCCAAAAACATAAACAACACTCAATAAATAAAATGAACAAACAAACGGACTAAAACACGCAGCAACGCATATTGTACATCGACGCACGGGACTTTTGACTC
ACAGACAATTCGAGAGTAGGGGGTCCACAGCACGCGATCGCGCGTAGAATGCGGAACCACACTACAAGATAAGTAAGACACATTATGACATGACTTGCGCCGAGTGAGTCTATTTAATCAACAAATGTAGTGATGTAAAGAAAAATACAACGCGTGTACACACACACACTATGGAATCATACATAAACACGTCGAGGGCG
ATCAAGGACTGTAAATGATACACACCGCGTATTTATAGGGAATCAACTACACAAACAGCGCACACTAACCGACTAATAGAGTTTGACGCACGCGGTGACACTCGCACAAAATAAAGTTTTTTACATATAAACGCAGAACAAGTTTTAGTCGCCGCGGAGAAATAACACAACAAAATCAGTATATACAAACGGGGGGACAC
CGGTAAAACTCACAAAGAAGCGACGCGAACACCACGTGATATATAGACATACAAATATACCACCGCAATGCGCACACACACACCTGTACAAACATGACAAGT